# Extracting Sentinel2 iamgery using Google Earth Engine
## for Amsterdam, The Hague, Rotterdam and Utrecht




## Initializing GEE API 

In [1]:
import ee

# Trigger the authentication flow.
ee.Authenticate()

try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise


KeyboardInterrupt



## Getting Data

In [2]:
#using ggemap python package for interactive mapping
import geemap

In [3]:
#Sentinel 2 cloud-free image in the Netherlands
collection = ('COPERNICUS/S2')
list_netherlands = list([(3.31, 50.80),(7.093, 50.80),(7.093, 53.52),(3.31, 53.52),(3.31, 50.80)])
area_netherlands = ee.Geometry.Polygon(list_netherlands)

In [16]:
#Gather images of Netherlands for 2021 and 2022 and with cloud percentage less than 5  

myCollection = ee.ImageCollection(collection) \
    .filterBounds(area_netherlands) \
    .filterDate('2021-01-01', '2022-12-31') \
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 5) \
    .sort('CLOUDY_PIXEL_PERCENTAGE', False)

In [17]:
# Total number of images in the Netherlands
listOfImages = myCollection.aggregate_array('system:index').getInfo()
print('Number of images in the collection: ', len(listOfImages))

Number of images in the collection:  730


In [30]:
# Reduce the Img collection by median.
img = myCollection.median()

In [19]:
# Display the result. 
vis_params = {"min": 0, 
              "max": 4000, 
              "bands": ["B4", "B3", "B2"]} 
Map = geemap.Map()
center = ee.Geometry.Point([5.29, 52.13]).getInfo()['coordinates']
Map.setCenter(center[0], center[1], 7)
Map.addLayer(img, vis_params, 'netherlands')
Map

Map(center=[52.13, 5.29], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [31]:
# Get information about the bands as a list. 
bandNames = img.bandNames() 
print('Band names: ', bandNames.getInfo()) # ee.List of band names 

Band names:  ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B9', 'B10', 'B11', 'B12', 'QA10', 'QA20', 'QA60']


In [32]:
# Get projection information from band 1. 
proj = img.projection() 
print('projection: ', proj.getInfo()) # ee.Projection object 

projection:  {'type': 'Projection', 'crs': 'EPSG:4326', 'transform': [1, 0, 0, 0, 1, 0]}


In [22]:
# Get spatial resolution (in meters) information from bands
bscale = img.select('B1').projection().nominalScale() 
print('Band 1 scale: ', bscale.getInfo()) # ee.Number 
bscale = img.select('B2').projection().nominalScale() 
print('Band 2 scale: ', bscale.getInfo()) # ee.Number 
bscale = img.select('B3').projection().nominalScale() 
print('Band 3 scale: ', bscale.getInfo()) # ee.Number
bscale = img.select('B4').projection().nominalScale() 
print('Band 4 scale: ', bscale.getInfo()) # ee.Number
bscale = img.select('B8').projection().nominalScale() 
print('Band 8 scale: ', bscale.getInfo()) # ee.Number

Band 1 scale:  111319.49079327357
Band 2 scale:  111319.49079327357
Band 3 scale:  111319.49079327357
Band 4 scale:  111319.49079327357
Band 8 scale:  111319.49079327357


In [42]:
# Select the desired bands
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

# Select the desired bands from the image
img_selectedbands = img.select(bands)

In [61]:
# Get information about the selected bands as a list. 
bandNames = img_selectedbands.bandNames() 
print('Band names: ', bandNames.getInfo()) # ee.List of band names 

Band names:  ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']


### Clip an image by region of interest

https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api#clip_an_image_by_a_region_of_interest

## Amsterdam

In [62]:
#Clip an image by region of interest

# Define the bounding box of Amsterdam
list_amsterdam = list([(4.728758, 52.278175),(5.079161, 52.278175),(5.079161, 52.431064),(4.728758, 52.431064),(4.728758, 52.278175)])
area_amsterdam = ee.Geometry.Polygon(list_amsterdam)

# Clip the image by Amsterdam area.
amsterdam = img_selectedbands.clip(area_amsterdam)


vis_params = {"min": 0, 
              "max": 4000, 
              "bands": ["B4", "B3", "B2"]} 
Map = geemap.Map()
center = ee.Geometry.Point([5.29, 52.13]).getInfo()['coordinates']
Map.setCenter(center[0], center[1], 7)
Map.addLayer(amsterdam, vis_params, 'amsterdam')
Map

Map(center=[52.13, 5.29], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children…

In [64]:
# Export Amsterdam image to GEOTIFF file with the pixel resolution of 10m*10m and with Dutch reference system 

#Amsterdam = amsterdam.reproject(crs='EPSG:28992', scale=10)
task = ee.batch.Export.image.toDrive(image=amsterdam,
                                     description='Amsterdam_Sen2',
                                     scale=10,
                                     region=area_amsterdam,
                                     fileNamePrefix='Amsterdam_Sen2',
                                     crs='EPSG:28992',
                                     fileFormat='GeoTIFF')
task.start()

In [67]:
task.status()

{'state': 'RUNNING',
 'description': 'Amsterdam_Sen2',
 'creation_timestamp_ms': 1681210033607,
 'update_timestamp_ms': 1681210053126,
 'start_timestamp_ms': 1681210053092,
 'task_type': 'EXPORT_IMAGE',
 'attempt': 1,
 'id': '3CHNUXK6GSGY5B6Z6C5IKIXX',
 'name': 'projects/earthengine-legacy/operations/3CHNUXK6GSGY5B6Z6C5IKIXX'}